In [1]:
import pdfplumber, camelot, pprint
import fitz #type: ignore
import pandas as pd
import numpy as np
import json, os, re

In [10]:
path = os.path.join(os.getcwd(),"pdfs","1736224555418.pdf")
path2 = os.path.join(os.getcwd(),"pdfs","Angel-One-Nifty-1D-Rate-Liquid-ETF-Growth.pdf")

In [8]:
all_data = []
with pdfplumber.open(path2) as doc:
    for i, page in enumerate(doc.pages):
        if i in [1,2,3]:
            text = page.extract_text()
            all_data.append(text)

In [ ]:
pprint.pprint(all_data)

In [11]:
scheme_tables = camelot.read_pdf(path2, pages="5,6,7,8,9", flavor='lattice')
dfs = all_data = pd.concat([table.df for table in scheme_tables], ignore_index=True)
dfs = dfs.drop(dfs.columns[0], axis=1)
dfs = dfs.applymap(lambda x: x.replace('\n', ' ') if isinstance(x, str) else x) #applies to dataframe not series
dfs.iloc[:, 0] = dfs.iloc[:, 0].replace(r'^\s*$', np.nan, regex=True)
dfs.iloc[:, 0] = dfs.iloc[:, 0].ffill() #fill corresponding structure
new_column_names = ['Title'] + [f'Data{i}' for i in range(1, dfs.shape[1])]
dfs.columns = new_column_names
# html = dfs.to_html("check.html")
data_cols = [col for col in dfs.columns if col != 'Title']
result_dict = {}
for title, group_df in dfs.groupby('Title', sort=False):
    title = re.sub(r"\s+"," ",title).strip()
    title = "_".join(title.lower().split(" "))
    
    current_title_values = []
    for index, row in group_df.iterrows():
        for col_name in data_cols:
            cell_value = row[col_name]
            if isinstance(cell_value, str) and cell_value.strip() != '':
                current_title_values.append(cell_value)
    result_dict[title] = current_title_values

C:\Users\rando\AppData\Local\Temp\ipykernel_9404\1361140556.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfs = dfs.applymap(lambda x: x.replace('\n', ' ') if isinstance(x, str) else x) #applies to dataframe not series


MANAGER

In [12]:
manager_tables = camelot.read_pdf(path2, pages="14,15", flavor='lattice')
dfs = all_data = pd.concat([table.df for table in manager_tables], ignore_index=True)
dfs = dfs.applymap(lambda x: x.replace('\n', ' ') if isinstance(x, str) else x)
fund_manager_data = {
    "fund_manager": [list(row) for _, row in dfs.iterrows()]
}

C:\Users\rando\AppData\Local\Temp\ipykernel_9404\2179127116.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfs = dfs.applymap(lambda x: x.replace('\n', ' ') if isinstance(x, str) else x)


In [13]:
result_dict.update(fund_manager_data)
with open("final.json","w+") as data:
    json.dump(result_dict,data)